In [102]:
import rpa as r
import pandas as pd
from RPA.Excel.Application import Application
from RPA.Email.ImapSmtp import ImapSmtp

In [103]:
class Solution():
    _UserEmail = ""
    _UserPass = ""
    _df = pd.DataFrame()
    def __init__(self,UserEmail,UserPass):
        self._UserEmail = UserEmail
        self._UserPass = UserPass
    def ParseValues(self):
        answers = {'(A) Дата':[], '(B) Курс':[], '(C)Изменение':[],'(D) Дата Евро':[], '(E) Курс Евро':[], '(F)Изменение Евро':[]}
        r.init()
        r.url('https://yandex.ru/news/quotes/2002.html')
        for i in range(2,12):
            for k in range(1,4):
                if k == 1:
                    answers['(A) Дата'].append(r.read('/html/body/div[3]/div[1]/div[1]/div/div/div[2]/div/div[2]/table/tbody/tr['+str(i)+']/td['+str(k)+']'))
                elif k == 2:
                    answers['(B) Курс'].append(r.read('/html/body/div[3]/div[1]/div[1]/div/div/div[2]/div/div[2]/table/tbody/tr['+str(i)+']/td['+str(k)+']').replace(',','.'))
                elif k == 3:
                    answers['(C)Изменение'].append(r.read('/html/body/div[3]/div[1]/div[1]/div/div/div[2]/div/div[2]/table/tbody/tr['+str(i)+']/td['+str(k)+']').replace(',','.'))
        r.url('https://yandex.ru/news/quotes/2000.html')
        for i in range(2,12):
            for k in range(1,4):
                if k == 1:
                    answers['(D) Дата Евро'].append(r.read('/html/body/div[3]/div[1]/div[1]/div/div/div[2]/div/div[2]/table/tbody/tr['+str(i)+']/td['+str(k)+']'))
                elif k == 2:
                    answers['(E) Курс Евро'].append(r.read('/html/body/div[3]/div[1]/div[1]/div/div/div[2]/div/div[2]/table/tbody/tr['+str(i)+']/td['+str(k)+']').replace(',','.'))
                elif k == 3:
                    answers['(F)Изменение Евро'].append(r.read('/html/body/div[3]/div[1]/div[1]/div/div/div[2]/div/div[2]/table/tbody/tr['+str(i)+']/td['+str(k)+']').replace(',','.'))
        self._df = pd.DataFrame(answers)
        self._df['(B) Курс'] = self._df['(B) Курс'].astype('float')
        self._df['(C)Изменение'] = self._df['(C)Изменение'].astype('float')
        self._df['(E) Курс Евро'] = self._df['(E) Курс Евро'].astype('float')
        self._df['(F)Изменение Евро'] = self._df['(F)Изменение Евро'].astype('float')
        self._df['(G) Деление'] = self._df['(B) Курс']/self._df['(E) Курс Евро']
        r.close()
    def WriteInExcel(self):
        print(self._df)
        app = Application()
        app.open_application(True)
        app.add_new_workbook()
        app.add_new_sheet('Лист1')
        app.set_active_worksheet(sheetname='Лист1')
        for i in range(len(self._df)):
            for num,k in enumerate(self._df.columns):
                if k=="(D) Дата Евро" or k=="(A) Дата":
                    app.write_to_cells(row = i+1, column = num+1, value = self._df[k][i])
                    continue
                if num>2 and num!=6:
                    app.write_to_cells(row = i+1, column = num+1, value = self._df[k][i], number_format='####0,00\€')
                    continue
                if num!=6:
                    app.write_to_cells(row = i+1, column = num+1, value = self._df[k][i], number_format='####0,00\$')
        app.save_excel_as('C:/Users/Esteras/Desktop/Hi.xlsx', autofit=True)
    def SendMessage(self,RecevierMail):
        try:
            mail = ImapSmtp(smtp_server="smtp.gmail.com", smtp_port=587)
            mail.authorize(account=self._UserEmail, password=self._UserPass)
            mail.send_message(
                sender=self._UserEmail,
                recipients=RecevierMail,
                subject="Message from RPA Python",
                body="Файл содержит "+str(len(self._df))+ " строк",
                attachments=["C:/Users/Esteras/Desktop/Hi.xlsx"]
            )
        except Exception:
            print("Произошла ошибка, проверьте пару логин пароль и настройки Gmail")

In [104]:
TaskSolution = Solution("tetstt887@gmail.com" , "qqwwee112233")

In [105]:
TaskSolution.ParseValues()

In [106]:
TaskSolution.WriteInExcel()

   (A) Дата  (B) Курс  (C)Изменение (D) Дата Евро  (E) Курс Евро  \
0  20.07.20   71.4025       -0.4700      20.07.20        81.6175   
1  17.07.20   71.8725        0.3425      17.07.20        82.2000   
2  16.07.20   71.5300        0.5400      16.07.20        81.4900   
3  15.07.20   70.9900        0.1250      15.07.20        80.9900   
4  14.07.20   70.8650       -0.1050      14.07.20        80.7500   
5  13.07.20   70.9700        0.2425      13.07.20        80.5075   
6  10.07.20   70.7275       -0.1875      10.07.20        79.8800   
7  09.07.20   70.9150       -0.1950      09.07.20        80.0000   
8  08.07.20   71.1100       -0.3625      08.07.20        80.5925   
9  07.07.20   71.4725       -0.3550      07.07.20        80.6300   

   (F)Изменение Евро  (G) Деление  
0            -0.5825     0.874843  
1             0.7100     0.874361  
2             0.5000     0.877776  
3             0.2400     0.876528  
4             0.2425     0.877585  
5             0.6275     0.881533  

In [107]:
TaskSolution.SendMessage("tetstt887@gmail.com")